In [37]:
import folium
import pandas as pd
import base64
from io import BytesIO

In [38]:
mapa = folium.Map(location=[-15.7836, -47.7750])

In [39]:
encoded = base64.b64encode(open('assets/Group.png', 'rb').read())
decoded = base64.b64decode(encoded)
icon_url = BytesIO(decoded)

In [44]:
df_hospitais = pd.read_csv('data/hospitais-brasilia.csv')
hospital_image_url = 'https://github.com/chris-redfield/hacksaude-2018/raw/master/assets/hostpital-building.png'
for index, row in df_hospitais.iterrows():
    
    icon_url = BytesIO(decoded)
    html = '<h5>' + row['nome'] + '</h5><hr style="margin-top:0px">' + '<img src="'+hospital_image_url \
    +'" style="float:left;padding-right:13px">'\
    'Total de profissionais: '\
    + str(row['qtd_profissionais']) + '<br>' + 'Atendimento: ' +row['atendimento']\
    + '<br>' + 'Telefone: ' + row['telefone']
    
    marker = folium.Marker([row['lat'],row['long']],
                  popup=html,
                  icon=folium.CustomIcon(icon_url,icon_size=(32,41),icon_anchor=(16,30)),
                 ).add_to(mapa)
    
    folium.CircleMarker([row['lat'],row['long']],
                    radius=row['qtd_profissionais']/150,
                    color='#3186cc',
                    fill=True,
                    fill_color='#3186cc',
                    fillOpacity= 0.8,
                   ).add_to(mapa)
mapa